In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle
import os
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


# SkillDev - Data Preprocessing & Clustering Model


In [5]:
class SkillDev:
    def __init__(self, file_path, n_clusters=3):
        print(f"Loading data from {file_path}...")
        self.df = pd.read_csv(file_path)
        self.n_clusters = n_clusters
        self.scaler = StandardScaler()
        self.kmeans = None
        self.features = None
        
    def preprocess_data(self):
        """Convert columns to numeric and prepare for clustering"""
        print("\n📊 Preprocessing data...")
        
        # Convert all columns to numeric
        for col in self.df.columns:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
        
        # Drop columns that are entirely NaN
        self.df = self.df.dropna(axis=1, how='all')
        
        # Drop rows with all NaN values
        self.df = self.df.dropna(how='all')
        
        # Get numeric features, excluding ID-like columns
        self.features = [col for col in self.df.select_dtypes(include=[np.number]).columns 
                        if col.upper() not in ['ID', 'INDEX']]
        
        # Drop features with too many missing values (>50%)
        threshold = len(self.df) * 0.5
        self.features = [col for col in self.features if self.df[col].isna().sum() < threshold]
        
        # Fill remaining missing values with median
        for col in self.features:
            median_val = self.df[col].median()
            if pd.isna(median_val):
                self.df[col].fillna(0, inplace=True)
            else:
                self.df[col].fillna(median_val, inplace=True)
        
        # Remove rows with any remaining NaN values in selected features
        self.df = self.df.dropna(subset=self.features)
        
        print(f"✅ Data preprocessed. Records: {len(self.df)}, Features: {len(self.features)}")
        return self.df
    
    def train_clusters(self):
        """Train K-Means clustering"""
        print(f"\n🤖 Training K-Means clustering with {self.n_clusters} clusters...")
        
        X = self.scaler.fit_transform(self.df[self.features])
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=42)
        self.df['cluster_id'] = self.kmeans.fit_predict(X)
        
        print(f"✅ Clustering complete!")
        
    def show_clusters(self):
        """Display cluster information"""
        print(f"\n📋 CLUSTER SUMMARY:")
        print(f"Total records: {len(self.df)}")
        print(f"\nCluster Distribution:")
        print(self.df['cluster_id'].value_counts().sort_index())
        
        print(f"\n📊 CLUSTER CENTERS:")
        for i, center in enumerate(self.kmeans.cluster_centers_):
            print(f"Cluster {i}: {center[:5]}...")  # Show first 5 features
        
        print(f"\n🎯 CLUSTER SAMPLES:")
        for cluster_id in range(self.n_clusters):
            cluster_data = self.df[self.df['cluster_id'] == cluster_id]
            print(f"\nCluster {cluster_id} - {len(cluster_data)} members")
            print(cluster_data[self.features[:5]].head(3))
    
    def save_model(self, model_path):
        """Save model to pickle"""
        os.makedirs(os.path.dirname(model_path) if os.path.dirname(model_path) else '.', exist_ok=True)
        with open(model_path, 'wb') as f:
            pickle.dump(self, f)
        print(f"\n✅ Model saved to {model_path}")


In [6]:
import os

file_name = 'LFS-2023.csv'

candidate_paths = [
    os.path.join('data', file_name),
    os.path.join('..', 'data', file_name),
    os.path.join(os.getcwd(), 'data', file_name),
    os.path.join(os.getcwd(), '..', 'data', file_name),
]

csv_file = next((p for p in candidate_paths if os.path.exists(p)), None)
if not csv_file:
    searched = "\n".join(candidate_paths)
    raise FileNotFoundError(f"Please place '{file_name}' in the data folder.\nSearched:\n{searched}")

# Initialize, preprocess, and train
system = SkillDev(csv_file, n_clusters=3)
system.preprocess_data()
system.train_clusters()
system.show_clusters()

# Save model
model_folder = os.path.join('..', 'model')
os.makedirs(model_folder, exist_ok=True)
pkl_file = os.path.join(model_folder, 'skilldev_model.pkl')
system.save_model(pkl_file)


Loading data from ..\data\LFS-2023.csv...

📊 Preprocessing data...
✅ Data preprocessed. Records: 18937, Features: 32

🤖 Training K-Means clustering with 3 clusters...
✅ Clustering complete!

📋 CLUSTER SUMMARY:
Total records: 18937

Cluster Distribution:
cluster_id
0     6638
1    11273
2     1026
Name: count, dtype: int64

📊 CLUSTER CENTERS:
Cluster 0: [ 0.         -0.01770159  0.01672604  0.00031657 -0.03451391]...
Cluster 1: [ 0.          0.00544404 -0.0142866  -0.00690652  0.01963138]...
Cluster 2: [0.         0.05477474 0.04854533 0.07372227 0.00787646]...

🎯 CLUSTER SAMPLES:

Cluster 0 - 6638 members
   YEAR  MONTH  SECTOR  DISTRICT  PSU
5  2023      1       1        11    1
6  2023      1       1        11    1
9  2023      1       1        11    1

Cluster 1 - 11273 members
   YEAR  MONTH  SECTOR  DISTRICT  PSU
0  2023      1       1        11    1
1  2023      1       1        11    1
2  2023      1       1        11    1

Cluster 2 - 1026 members
    YEAR  MONTH  SECTOR  DISTR